In [3]:
test = """
broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a
"""

test2 = """
broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output
"""

# Flip-flop modules (prefix %) are either on or off; they are initially off. 
# If a flip-flop module receives a high pulse, it is ignored and nothing happens.
# However, if a flip-flop module receives a low pulse, it flips between on and off.
# If it was off, it turns on and sends a high pulse.
# If it was on, it turns off and sends a low pulse.

# Conjunction modules (prefix &) remember the type of the most recent pulse received from each of their connected input modules; 
# they initially default to remembering a low pulse for each input.
# When a pulse is received, the conjunction module first updates its memory for that input.
# Then, if it remembers high pulses for all inputs, it sends a low pulse; otherwise, it sends a high pulse.

# There is a single broadcast module (named broadcaster).
# When it receives a pulse, it sends the same pulse to all of its destination modules.

class Module:
    def __init__(self, name, destinations, inputs):
        self.name = name
        self.destinations = destinations
        self.inputs = inputs

class FlipFlop(Module):
    def __init__(self, name, destinations, inputs):
        super().__init__(name, destinations, inputs)
        self.on = False

    def high_pulse(self, input):
        return None

    def low_pulse(self, input):
        if self.on:
            output = 'low_pulse'
        else:
            output = 'high_pulse'
        
        self.on = not self.on
        return output        

class Conjunction(Module):
    def __init__(self, name, destinations, inputs):
        super().__init__(name, destinations, inputs)
        self.last_inputs = {input: 'low_pulse' for input in inputs}

    def high_pulse(self, input):
        self.last_inputs[input] = 'high_pulse'
        return self._pulse()

    def low_pulse(self, input):
        self.last_inputs[input] = 'low_pulse'
        return self._pulse()

    def _pulse(self):
        if all(input == 'high_pulse' for input in self.last_inputs.values()):
            return 'low_pulse'
        else:
            return 'high_pulse'

class Broadcaster(Module):
    def __init__(self, name, destinations, inputs):
        super().__init__(name, destinations, inputs)

    def high_pulse(self, input):
        return 'high_pulse'

    def low_pulse(self, input):
        return 'low_pulse'

def parse_module(data):
    name, destinations = data.split('->')
    destinations = destinations.split(',')
    if name.startswith('%'):
        prefix = '%'
    elif name.startswith('&'):
        prefix = '&'
    else:
        prefix = None
    
    return name.strip('%&'), {'prefix': prefix, 'destinations': destinations}


from collections import defaultdict

def build_input_map(output_map):
    input_map = defaultdict(list)
    for name, data in output_map.items():
        for destination in data['destinations']:
            input_map[destination].append(name)
    return input_map

def build_module_map(output_map, input_map):
    module_map = {}
    class_map = {
            '%': FlipFlop,
            '&': Conjunction,
            None: Broadcaster
        }
    for name, data in output_map.items():
        prefix = data['prefix']
        destinations = data['destinations']
        inputs = input_map[name]

        klass = class_map[prefix]
        module_map[name] = klass(name, destinations, inputs)

    return module_map

In [4]:
from collections import deque, namedtuple, Counter

def process_queue(pulse_queue):
    while pulse_queue:
        pulse = pulse_queue.popleft()
        source, current, input_intensity = pulse
        
        module = module_map.get(current)
        if not module:
            continue
        intensity = getattr(module, input_intensity)(source)
        if not intensity:
            continue
    
        for destination in module.destinations:
            #print(current, f'-{intensity}->', destination)
            new_pulse = Pulse(source=current, dest=destination, intensity=intensity)
            pulse_queue.append(new_pulse)  
            counter[intensity] += 1

# data = test.strip().splitlines()

file_path = 'day_20_input.txt'
with open(file_path, 'r') as file:
    data = [line.strip() for line in file]

output_map = dict(parse_module(module.replace(" ", "")) for module in data)
input_map = build_input_map(output_map)
module_map = build_module_map(output_map, input_map)
module_map

Pulse = namedtuple('Pulse', ['source', 'dest', 'intensity'])
pulse = Pulse(source=None, dest='broadcaster', intensity='low_pulse')
pulse_queue = deque()
counter = Counter()

for x in range(1000):
    pulse_queue.append(pulse)
    counter['low_pulse'] += 1
    process_queue(pulse_queue)

counter

Counter({'high_pulse': 39083, 'low_pulse': 17061})

In [5]:
counter['low_pulse'] * counter['high_pulse']

666795063

In [37]:
from collections import deque, namedtuple, Counter

def process_queue(pulse_queue):
    counter = Counter()
    while pulse_queue:
        pulse = pulse_queue.popleft()
        source, current, input_intensity = pulse
        
        module = module_map.get(current)
        if not module:
            continue
        intensity = getattr(module, input_intensity)(source)
        if not intensity:
            continue
    
        for destination in module.destinations:
            new_pulse = Pulse(source=current, dest=destination, intensity=intensity)
            pulse_queue.append(new_pulse)  
            counter[f'{destination}_{intensity}'] += 1
    
    return counter


# data = test.strip().splitlines()

def single_low_pulse(counter):
    return counter['rx_low_pulse'] == 1 and counter ['rx_high_pulse'] == 0


file_path = 'day_20_input.txt'
with open(file_path, 'r') as file:
    data = [line.strip() for line in file]

output_map = dict(parse_module(module.replace(" ", "")) for module in data)
input_map = build_input_map(output_map)
module_map = build_module_map(output_map, input_map)
module_map

Pulse = namedtuple('Pulse', ['source', 'dest', 'intensity'])
pulse = Pulse(source=None, dest='broadcaster', intensity='low_pulse')
pulse_queue = deque()
counter = Counter()
count = 0

while not single_low_pulse(counter):
    count += 1
    pulse_queue.append(pulse)
    counter = process_queue(pulse_queue)
    for key, value in module_map['xl'].last_inputs.items():
        if value == 'high_pulse':
            print('count:', count, 'module', key)
    if count > 10:
        break

count

count: 1 module zp
count: 2 module zp
count: 3 module zp
count: 4 module zp
count: 5 module zp
count: 6 module zp
count: 7 module zp
count: 8 module zp
count: 9 module zp
count: 10 module zp
count: 11 module zp


11